<a href="https://colab.research.google.com/github/minjae960/CNN_Exercise/blob/main/Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import random

#Image Samples Pre-processing

In [ ]:
# Download Image Samples from Github
!git clone https://github.com/minjae960/CNN_Exercise.git

In [ ]:
DIR = '/content/CNN_Exercise/CNT Sample'
CATEGORIES = ['No', 'Sonic']

In [ ]:
CNT_IMG = []

for category in CATEGORIES:
    Path = os.path.join(DIR, category)

    for file in os.listdir(Path):
        # Change Image to Numpy array in grayscale
        img_array = cv2.imread(os.path.join(Path, file), cv2.IMREAD_GRAYSCALE)
        # Save image array, category as 0 or 1, and file name to CNT_IMG
        CNT_IMG.append([img_array, CATEGORIES.index(category), file])

In [ ]:
# Show Image Samples
for i, data in enumerate(CNT_IMG):
    img_array = data[0]
    name = data[2]

    plt.rcParams["figure.figsize"] = (20,7)

    plt.subplot(2,4,i+1)
    plt.imshow(img_array, cmap='gray')
    plt.title(name)
    plt.show

In [ ]:
# Four splits in row and six splits in column -> 24 small samples from one 1080 x 1920 image
n_row = 4
n_col = 6

# Resize image to 256 x 256 pixels
img_size = 256

SMALL_IMG = []

for img_array, label, name in CNT_IMG:
    for i in range(n_row):
        for k in range(n_col):
            height = int(img_array.shape[0]/n_row)
            width = int(img_array.shape[1]/n_col)

            small_img = img_array[i*height:(i+1)*height, k*width:(k+1)*width] # Split -> 270 x 320

            small_img = cv2.resize(small_img, (img_size, img_size)) # Resize -> 256 x 256

            # Normalization
            small_img = small_img / 255.0 

            SMALL_IMG.append([small_img, label, name])

In [ ]:
# Show 10 Random Images out of 192(= 8 x 24) Small Image Samples

RANDOM_IMG = random.sample(SMALL_IMG, 10)

i=1

for img_array, label, name in RANDOM_IMG:
    

    plt.rcParams["figure.figsize"] = (20,7)

    plt.subplot(2,5,i)
    plt.imshow(img_array, cmap='gray')
    plt.title(name)
    plt.show

    i+=1

In [ ]:
# Shuffle images to be trained well

random.shuffle(SMALL_IMG)

In [ ]:
# 160 Samples -> Train set
# 32 Samples -> Test set

Train_Data = SMALL_IMG[:160]
Test_Data = SMALL_IMG[160:]

In [ ]:
x_train = []
y_train = []

for img_array, label, name in Train_Data:
    x_train.append(img_array)
    y_train.append(label)

# Change List to Numpy
x_train = np.array(x_train).reshape(160, img_size, img_size, 1) # (instances, height, width, depth)
y_train = np.array(y_train)

print(x_train.shape, y_train.shape)

#CNN Model

In [ ]:
# Build TensorFlow model -> CNN for Logistic Regression -> 2 Class Classification
tf.model = tf.keras.Sequential()

# Conv -> ReLU -> Conv -> ReLU -> Maxpooling
tf.model.add(tf.keras.layers.Conv2D(filters=64, input_shape=(img_size,img_size,1), kernel_size=(3,3), # filters = num of filters / kernel_size = (filter height, filter width)
                                    kernel_initializer='glorot_normal',activation='relu', padding='same')) # padding = 'same' -> size of input = size of output
tf.model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), 
                                    kernel_initializer='glorot_normal',activation='relu', padding='same')) 
tf.model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2))) # pool_size = (filter_size, stride)

# Conv -> ReLU -> Conv -> ReLU -> Maxpooling
tf.model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), kernel_initializer='glorot_normal', 
                                    activation='relu', padding='same'))
tf.model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), kernel_initializer='glorot_normal', 
                                    activation='relu', padding='same'))
tf.model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

# Fully Connected
tf.model.add(tf.keras.layers.Flatten()) # 3D data -> 1D data
tf.model.add(tf.keras.layers.Dense(units=1, kernel_initializer='glorot_normal', activation='sigmoid'))

tf.model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.001), metrics=['accuracy'])
tf.model.summary()

# Train
tf.model.fit(x_train, y_train, validation_split=0.1, batch_size=100, epochs=40)

#Test CNN Model

In [ ]:
# Test set
x_test = []
y_test = []

for img_array, label, name in Test_Data:
    x_test.append(img_array)
    y_test.append(label)

x_test = np.array(x_test).reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)

In [ ]:
# Accuracy
Train_Accuracy = tf.model.evaluate(x_train, y_train)
Test_Accuracy = tf.model.evaluate(x_test, y_test)

In [ ]:
# Predict Test Set
Prediction = []

for pred in tf.model.predict(x_test):
    if pred > 0.5:
        Prediction.append('Sonic')
    else:
        Prediction.append('No')

In [ ]:
# Show First 10 Test Image Samples and Predictions

i = 1

for img_array, label, name in Test_Data[:10]:
    
    plt.rcParams["figure.figsize"] = (20,7)

    plt.subplot(2,5,i)
    plt.imshow(img_array, cmap='gray')
    plt.title(name)
    plt.show

    i+=1

print(np.array(Prediction[:10]).reshape(2,5))